# Проектная работа

## Загрузка и обработка текстовых данных

In [ ]:
import os

dir_export = 'export'
os.makedirs(dir_export, exist_ok=True)

dir_import = 'import'
os.makedirs(dir_import, exist_ok=True)

### Импорт библиотек

In [ ]:
from collections import defaultdict
import csv
from datetime import datetime
import string
import re
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from pymystem3 import Mystem

### Чтение текстовых данных в групппированный словарь сообщений

```
[
  'id пользователя',
  [
    {
      'text': Текст сообщения,
      'date': Дата сообщения
    },
      ...
  ]
  ...
]
```

In [ ]:
def read_and_group_messages_by_user(csv_filename):
    user_messages = defaultdict(list)
    user_name = dict()

    with open(csv_filename, mode='r', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            user_id = row['user_id']
            message = row['message']
            # Преобразование строки в объект datetime
            date = datetime.fromisoformat(row['date']).strftime('%d.%m.%Y %H:%M:%S')
            user_messages[user_id].append({
                'text': message,
                'date': date,
            })
            user_name[user_id] = {
                'username': row['username'],
                'fullname': row['fullname'],
            }
    
    return user_messages, user_name
    
messages_filename = os.path.join(dir_import, "messages.csv")
grouped_messages, users = read_and_group_messages_by_user(messages_filename)

### Вывод ID пользователей

In [ ]:
for user_id in grouped_messages.keys():
    print(f"{user_id}:\t@{users[user_id]['username'] or '-'} ({users[user_id]['fullname'] or '-'})")

### Выбор пользователя для тестов

In [ ]:
user_id = '536537422'
user = f"@{users[user_id]['username'] or '-'} ({users[user_id]['fullname'] or '-'})"

### Вывод всех сообщений пользователя

In [ ]:
print(f"Сообщения от пользователя ID {user_id} ({user}): ")
for message in grouped_messages[user_id]:
    print(f" - {message['date']} {message['text']}")

### Обработка текста

```
[
  'id пользователя',
  [
    {
      'text': Текст сообщения,
      'date': Дата сообщения,
      'tokens': Обработанный текст
    },
      ...
  ]
  ...
]
```

In [ ]:
def text_processing(text):
    translator = str.maketrans('', '', string.punctuation)
    stop_set = set(stopwords.words('russian'))
    mystem = Mystem()
    
    words = text.lower().translate(translator).split()
    words_onlyrus = re.findall(r'\b[а-яА-Я]+\b', text.lower())
    tokens = list(filter(lambda x: x not in {' ', '\n'} | stop_set, words_onlyrus))
    # tokens = mystem.lemmatize(' '.join(tokens))
    return tokens

for user_messages in grouped_messages.values():
    print(f'Обработка сообщений пользователя {user_id}:')
    for message in tqdm(user_messages):
        message['tokens'] = text_processing(message['text'])

print(f"Сообщения от пользователя ID {user_id} ({user}): ")
for message in grouped_messages[user_id]:
    print(f" - {message['date']} {message['tokens']}")

### Объединение всех сообщений каждого пользователя

In [ ]:
user_all_messages = {}
for user, user_messages in grouped_messages.items():
    user_all_messages[user] = " ".join([" ".join(msg['tokens']) for msg in user_messages])

print(f"Текст сообщений от пользователя ID {user_id} ({user}): ")
print(user_all_messages[user_id])

## Частота вхождения слов в текст

### Импорт библиотек

In [ ]:
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
from IPython.display import display, HTML
from wordcloud import WordCloud

### Выбор пользователя

In [ ]:
user_id = '536537422'
user = f"@{users[user_id]['username'] or '-'} ({users[user_id]['fullname'] or '-'})"
text_user = user_all_messages[user_id]

### Нахождение количества слов в тексте

In [ ]:
blob = TextBlob(text_user)
items = blob.word_counts.items()
sorted_items = sorted(items, key=itemgetter(1), reverse=True)

### Вывод частоты слов в тексте в виде таблицы

In [ ]:
df = pd.DataFrame(sorted_items, columns=['word', 'count'])
export_xlsx = os.path.join(dir_export, f"words-{user_id}.xlsx")
df.to_excel(export_xlsx)
print(f"Таблица частоты слов пользователя с ID {user_id} ({user}): ")
display(HTML(df.to_html(index=False)))

### Вывод частоты слов в тексте в виде гистограммы

In [ ]:
axes = df[:10].plot.bar(x='word', y='count', legend=False)
# Добавление заголовка и меток осей
axes.set_title('Частота слов', fontsize=16)
axes.set_xlabel('Слова', fontsize=14)
axes.set_ylabel('Количество', fontsize=14)
# Сохранение диаграммы
export_plt = os.path.join(dir_export, f"words-{user_id}.png")
plt.savefig(export_plt)
# Вывод диаграммы на экран
plt.show()

### Создание словарного облака

In [ ]:
# Создание объекта WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white')
wordcloud = wordcloud.generate(list(user_all_messages.values())[1])

# Отображение облака слов
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

## Анализ тональности текста

### Импорт библиотек

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers import pipeline

import spacy
from nltk import Tree
from spacy import displacy
import ru_core_news_lg

### Составление набора топ-100 слов пользователя

In [ ]:
top_words = " ".join(list(map(lambda x: x[0], sorted_items))[:100])
print(top_words)

### Анализ тональности
https://huggingface.co/blanchefort/rubert-base-cased-sentiment

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)
labels = ['Нейтральный', 'Позитивный', 'Негативный']

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    return predicted

probabilities = predict(top_words)
for label, prob in zip(labels, probabilities[0]):
    print(f"{label}: {prob:.4f}")

### Анализ токсичности
https://huggingface.co/IlyaGusev/rubertconv_toxic_clf

In [ ]:
model_name = "IlyaGusev/rubertconv_toxic_clf"
pipe = pipeline("text-classification", model=model_name, tokenizer=model_name, framework="pt") 
pipe([top_words])

## Дополнительные задачи

Другие модели для анализа текста можно посмотреть на сайте:
https://huggingface.co/models?pipeline_tag=text-classification&language=ru&sort=trending

### Текст для тестов

In [ ]:
import_text = os.path.join(dir_import, "new.txt")
with open(import_text, 'r', encoding='utf-8') as file:
    content = file.read()
print(content)

### Резюме статьи
https://huggingface.co/utrobinmv/t5_summary_en_ru_zh_base_2048

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'utrobinmv/t5_summary_en_ru_zh_base_2048'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

prefix = 'summary: '
src_text = prefix + content
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

# text brief summary generate
prefix = 'summary brief: '
src_text = prefix + content
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

# text big summary generate
prefix = 'summary big: '
src_text = prefix + content
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)